In [ ]:
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

from tensorflow.keras.datasets import cifar10

from tensorflow import device
from tensorflow.keras import regularizers
from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam,Adadelta
from tensorflow.keras.layers import Dense,Conv2D,Activation,BatchNormalization,MaxPooling2D,Dropout,Flatten,Input,concatenate,GlobalAveragePooling2D,GlobalMaxPooling2D                                                                                                                                                                                               

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import backend as K

In [ ]:
(X_train,Y_train),(X_test,Y_test) = cifar10.load_data()
X_train = (X_train.astype("float32")/255)-0.5
X_test = (X_test.astype("float32")/255)-0.5
Y_train = to_categorical(Y_train,10)
# Y_test = to_categorical(Y_test,10)
x_train,x_val,y_train,y_val = train_test_split(X_train,Y_train,test_size =0.2, random_state = 10)

In [ ]:
print("train size:",len(x_train))
print("validation size:",len(x_val))
print("test size:",len(X_test))
print("X shape:",X_train.shape[1:])

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
    rotation_range=15,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.15
    )
datagen.fit(X_train)

In [ ]:
inp_shape = x_train.shape[1:]
reg  = None
reg_l1 = regularizers.l1()
reg_l2 = regularizers.l2()
ks = 64
drop_size = 0.1
act = "relu"
from keras.callbacks import LearningRateScheduler
def lr_schedule(epoch):
    lrate = 0.001
    if epoch > 20:
        lrate = 0.0005
    if epoch > 40:
        lrate = 0.00025
    if epoch > 50:
        lrate = 0.0001
    return lrate

In [ ]:
i = Input(inp_shape)

#32----------------------------------------------------------------------------------
t11 = Conv2D(ks,(2,2),kernel_regularizer = reg,padding = "same",activation = act)(i)
t11 = BatchNormalization()(t11)
t1i  = concatenate((i,t11))

t12 = Conv2D(ks*2,(2,2),kernel_regularizer = reg,padding = "same",activation = act)(t1i)
t12 = BatchNormalization()(t12)
t2i = concatenate((i,t11,t12))

t13 = Conv2D(ks*4,(2,2),kernel_regularizer = reg,padding = "same",activation = act)(t2i)
t13 = BatchNormalization()(t13)
t13 = MaxPooling2D(2,2)(t13)
t13 = Dropout(drop_size*2)(t13)
#16----------------------------------------------------------------------------------
t21 = Conv2D(ks*2,(3,3),kernel_regularizer = reg,padding = "same",activation = act)(t13)
t21 = BatchNormalization()(t21)
t2i2  = concatenate((t13,t21))
t22 = Conv2D(ks*4,(3,3),kernel_regularizer = reg,padding = "same",activation = act)(t2i2)
t22 = BatchNormalization()(t22)
t2i3 = concatenate((t13,t21,t22))
t23 = Conv2D(ks*8,(3,3),kernel_regularizer = reg,padding = "same",activation = act)(t2i3)
t23 = BatchNormalization()(t23)
t23 = MaxPooling2D(2,2)(t23)
t23 = Dropout(drop_size*2)(t23)
#8--------------------------------------------------------------------------------------
t31 = Conv2D(ks*2,(3,3),kernel_regularizer = reg,padding = "same",activation = act)(t23)
t31 = BatchNormalization()(t31)
t3i2  = concatenate((t23,t31))
t32 = Conv2D(ks*4,(3,3),kernel_regularizer = reg,padding = "same",activation = act)(t3i2)
t32 = BatchNormalization()(t32)
t3i3 = concatenate((t23,t31,t32))
t33 = Conv2D(ks*8,(3,3),kernel_regularizer = reg,padding = "same",activation = act)(t3i3)
t33 = BatchNormalization()(t33)
t33 = MaxPooling2D(2,2)(t33)
t33 = Dropout(drop_size*2)(t33)
#4------------------------------------------------------------------------------------
# output = concatenate((t12,t))
# output = BatchNormalization()(output)
# output = GlobalAveragePooling2D()(t6)
# output = GlobalMaxPooling2D()(t6)
output = Flatten()(t33)
# output = Dropout(drop_size*4)
output = Dense(16*ks,kernel_regularizer = reg,activation = act)(output)
output = BatchNormalization()(output)
output = Dropout(2*drop_size)(output)
output = Dense(16*ks,kernel_regularizer = reg,activation = act)(output)
output = BatchNormalization()(output)
output = Dropout(2*drop_size)(output)
output = Dense(16*ks,kernel_regularizer = reg,activation = act)(output)
output = BatchNormalization()(output)
output = Dropout(2*drop_size)(output)

output = Dense(10,activation='softmax')(output)
model = Model(i,output)

model.compile(optimizer ="adam",
                     loss = "categorical_crossentropy",
                      metrics = ["accuracy"])

In [ ]:
model.summary()

In [ ]:
# history = model.fit_generator(datagen.flow(x_train,y_train),validation_data =(x_val,y_val),epochs = 80, callbacks=[LearningRateScheduler(lr_schedule)])                             

In [ ]:
# plt.plot(history.history['accuracy'])
# plt.plot(history.history['val_accuracy'])
# plt.title('model accuracy')
# plt.ylabel('accuracy')
# plt.xlabel('epoch')
# plt.legend(['train', 'test'], loc='upper left')
# plt.show()

In [ ]:
# plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss'])
# plt.title('model loss')
# plt.ylabel('loss')
# plt.xlabel('epoch')
# plt.legend(['train', 'test'], loc='upper left')
# plt.show()

In [ ]:
model.fit(datagen.flow(X_train,Y_train),epochs =65,callbacks=[LearningRateScheduler(lr_schedule)])

In [ ]:
y_pred = np.argmax(model.predict(X_test),axis = 1)
print(accuracy_score(y_pred,Y_test))